# Download the data

In [0]:
import os
from getpass import getpass
user = getpass('Kaggle Username: ')
key = getpass('Kaggle API key: ')

if '.kaggle' not in os.listdir('/root'):
    !mkdir ~/.kaggle
!touch /root/.kaggle/kaggle.json
!chmod 666 /root/.kaggle/kaggle.json
with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write('{"username":"%s","key":"%s"}' % (user, key))
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!mkdir data
%cd data
!kaggle competitions download -c dreem-sleep-stages
!unzip test.h5
!unzip train.h5
%cd ..

#Beginning of the code

In [0]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import csv
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from tqdm import tqdm
from scipy import signal

### Importation of the data

In [0]:
data = h5py.File("data/train.h5","r")

In [0]:
num_samples = 0

t1 = time.time()
#data
if num_samples:
    accX = data['accelerometer_x'][:num_samples]
    accY = data['accelerometer_y'][:num_samples]
    accZ = data['accelerometer_z'][:num_samples]
    eeg1 = data['eeg_1'][:num_samples]
    eeg2 = data['eeg_2'][:num_samples]
    eeg3 = data['eeg_3'][:num_samples]
    eeg4 = data['eeg_4'][:num_samples]
    eeg5 = data['eeg_5'][:num_samples]
    eeg6 = data['eeg_6'][:num_samples]
    eeg7 = data['eeg_7'][:num_samples]
    pulse = data['pulse_oximeter_infrared'][:num_samples]
else: 
    accX = data['accelerometer_x']
    accY = data['accelerometer_y']
    accZ = data['accelerometer_z']
    eeg1 = data['eeg_1']
    eeg2 = data['eeg_2']
    eeg3 = data['eeg_3']
    eeg4 = data['eeg_4']
    eeg5 = data['eeg_5']
    eeg6 = data['eeg_6']
    eeg7 = data['eeg_7']
    pulse = data['pulse_oximeter_infrared']
print(time.time()-t1)

if num_samples == 0:
    num_samples = accX.shape[0]

#label
Y = -1*np.ones(num_samples).astype(int)
with open('data/train_y.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    csv_reader.__next__()
    for i,row in enumerate(csv_reader):
        if i == num_samples:
            break
        Y[i] = int(row[1])
print(time.time()-t1)



0.005113124847412109
0.05380892753601074


In [0]:
450*11

4950

### Preprocessing the data

In [0]:
def filt_band(s,w):
    fs = s.shape[0]/30
    fso2 = fs/2
    b,a = signal.iirfilter(4, Wn = [w[0]/fso2,w[1]/fso2],btype = "band")
    return signal.lfilter(b,a,s)

def filt_low(s,w):
    fs = s.shape[0]/30
    fso2 = fs/2
    t1 = time.time()
    b,a = signal.iirfilter(4,Wn = w/fso2,btype = "lowpass")
    return signal.lfilter(b,a,s)

def filt_high(s,w):
    fs = s.shape[0]/30
    fso2 = fs/2
    b,a = signal.iirfilter(4,Wn = w/fso2,btype = "highpass")
    return signal.lfilter(b,a,s)

In [0]:
from IPython.display import clear_output

def preproc(signal):
    P = np.zeros((3,signal.shape[0]))
    P[0] = filt_low(signal,10)
    P[1] = filt_band(signal,[10,20])
    P[2] = filt_high(signal,20)
    #P[3] = filt_band(signal,[13,22])
    #P[4] = filt_high(signal,22)
    return P

def proc(signals, viz = False):
    n = signals[0].shape[0]
    delta = np.zeros((7,n,signals[0].shape[1]))
    theta = np.zeros((7,n,signals[0].shape[1]))
    alpha = np.zeros((7,n,signals[0].shape[1]))
    #beta = np.zeros((7,n,signals[0].shape[1]))
    #gamma = np.zeros((7,n,signals[0].shape[1]))
    #print("Matrixes loaded")
    t1 = time.time()
    for j, signal in enumerate(signals):
        signal = signal[:signal.shape[0]]
        for i,s in enumerate(signal):
            if i%100 ==1 and viz:
                clear_output()
                t2 = round(time.time()-t1)
                print("Time: %s s / %s s. Time for 1 filter: %s" % (t2, round(num_samples*7*t2/(i+num_samples*j)), (7*t2/(i+num_samples*j))))
                print(j,round(100*(i+j*num_samples)/(num_samples*7),1),"%")
            delta[j,i],theta[j,i],alpha[j,i] = preproc(s)
            #,beta[j,i],gamma[j,i] = preproc(s)
            
    return delta,theta,alpha#,beta,gamma

In [0]:
signals = [eeg1,eeg2,eeg3,eeg4,eeg5,eeg6,eeg7]
P = proc(signals,True)

num_proc = len(P)

Time: 322 s / 322 s. Time for 1 filter: 0.008412488103457929
6 100.0 %


### Feature extraction

In [0]:
def mmd(signal,lambd = -1):
    samples = signal.shape[1]
    if lambd == -1:
        if samples < 10000: lambd = 100
        else: lambd = int(10**(np.log10(samples)-1))
    mmd = np.zeros((signal.shape[0],int(samples/lambd)))
    for i in range(int(samples/lambd)):
        temp = signal[:,i*lambd:(i+1)*lambd]
        ymax = temp.max(1)
        ymin = temp.min(1)
        tmax = temp.argmax(1)
        tmin = temp.argmin(1)
        mmd[:,i] = np.sqrt((ymax-ymin)**2+(tmax-tmin)**2)
    return np.abs(mmd).sum(1)

  
def esis(signal, f,lambd = -1):
    samples = signal.shape[1]
    if lambd == -1:
        if samples < 10000: lambd = 100
        else: lambd = int(10**(np.log10(samples)-1))
    return np.abs(signal).sum() * f*lambd
    


def base_extract(signals):
    D = np.zeros((signals[0].shape[0],4*11))
    for i,s1 in tqdm(enumerate(signals)):
        s = np.copy(s1[:s1.shape[0]])
        D[:,4*i] = s.mean(1)
        D[:,4*i +1] = np.abs(s).mean(1)
        D[:,4*i +2] = np.abs(s).max(1)
        D[:,4*i +3] = np.abs(s).min(1)
    return D

def feat_extract(signals, do_pca = -1):
    nb_feat = int(5*4+7*3*2)
    D = np.zeros((signals[0].shape[0],nb_feat))
    i = 0
    num_s = 0
    while i < D.shape[1] and num_s < len(signals):
        s = np.copy(signals[num_s]); s = s[:s.shape[0]]
        if num_s < 4:
            D[:,i] = s.mean(1) ; i+=1
            D[:,i] = np.abs(s).mean(1) ; i+=1
            D[:,i] = np.abs(s).max(1) ; i+=1
            D[:,i] = np.abs(s).min(1) ; i+=1
            D[:,i] = s.std(1) ; i+=1
        else:
            D[:,i] = mmd(s) ; i+= 1
            if num_s in range(4,4+7): f = 5
            if num_s in range(4+7,4+2*7): f = 15
            if num_s in range(4+2*7,4+3*7): f = 25
            D[:,i] = esis(s,f) ; i+= 1
        num_s += 1
    
    pca = None
    if do_pca != -1:
        pca = PCA()
        pca.fit(D)
        D = pca.transform(D)[:,pca.explained_variance_ratio_.cumsum() < do_pca]
    return D, pca

In [0]:
t1 = time.time()

signals = [accX,accY,accZ,pulse]
for i in range(3):
    for j in range(7):
        signals.append(P[i][j])
D, pca = feat_extract(signals,-1)
print(time.time()-t1)
print(D.shape)

35.499258041381836
(38289, 62)


### Separation training - testing set

In [0]:
num_train = int(num_samples/2)
num_test = num_samples-num_train

#D, Y = shuffle(D,Y)

Dtr = D[:num_train]
Dtest = D[num_train:]
Ytr = Y[:num_train]
Ytest = Y[num_train:]


### Model

In [0]:
##define the model

class model(RandomForestClassifier):
    
    def __init__(self,ntree = 100):
        RandomForestClassifier.__init__(self,n_estimators=ntree)

In [0]:
#random forest
t1 = time.time()

rf = model(100)
rf.fit(Dtr,Ytr)
print(time.time()-t1)

Ypred = rf.predict(Dtest)
print(time.time()-t1)

10.377704858779907
10.861618995666504


In [0]:
Yv = rf.predict(Dtr)

In [0]:
#svm
t1 = time.time()

svmcl = svm.SVC()
svmcl.fit(Dtr,Ytr)
print(time.time()-t1)

Ypred = svmcl.predict(Dtest)
print(time.time()-t1)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


140.75638341903687
162.14497423171997


0.018935680389404297


In [0]:
#compute the mean f1-score
def f1score(Ypred,Ytest):
    f1 = -1*np.ones(5)
    f1score = -1
    for i in range(5):
        if (Ypred == i).sum():
            tp = np.sum((Ypred == Ytest)[Ypred == i])
            fp = np.sum((Ypred != Ytest)[Ypred == i])
            fn = np.sum((Ypred != Ytest)[Ytest == i])
            p = tp/(tp + fp)
            r = tp/(tp + fn)
            if p+r != 0: f1[i] = 2* (p*r)/(p+r)
            else: f1[i] = 0

    return f1[f1 != -1].mean()

#cross validation
def crossv(model,data,y):
    f1sc = np.zeros(10)
    indx = int(num_samples/10)
    data, y = shuffle(data,y)
    for i in tqdm(range(10)):
        model.__init__()
        model.fit(np.concatenate((data[:i*indx],data[(i+1)*indx:])),np.concatenate((y[:i*indx],y[(i+1)*indx:])))
        f1sc[i] = f1score(model.predict(data[i*indx:(i+1)*indx]),y[i*indx:(i+1)*indx])
    return f1sc

In [0]:
f1score(Ypred,Ytest)

0.5160980281365399

In [0]:
crossv(rf,D,Y)

100%|██████████| 10/10 [03:34<00:00, 21.49s/it]


array([0.54549814, 0.52614041, 0.54597001, 0.5436521 , 0.54415459,
       0.54011176, 0.54779453, 0.54995826, 0.54992048, 0.54646251])

### Writing the submission file

In [0]:
#Import the test data
data_test = h5py.File("data/test.h5",'r')

accX_val = data_test['accelerometer_x']
accY_val = data_test['accelerometer_y']
accZ_val = data_test['accelerometer_z']
eeg1_val = data_test['eeg_1']
eeg2_val = data_test['eeg_2']
eeg3_val = data_test['eeg_3']
eeg4_val = data_test['eeg_4']
eeg5_val = data_test['eeg_5']
eeg6_val = data_test['eeg_6']
eeg7_val = data_test['eeg_7']
pulse_val = data_test['pulse_oximeter_infrared']

In [0]:
num_val = accX_val.shape[0]
num_val

37439

In [0]:

#training the model on all the training data
model_final = model()

t1 = time.time()
model_final.fit(D,Y)
print("Training time: ",time.time()-t1)

Y_val = np.zeros(num_val)
pack = 100

for j in range(num_val%pack):
  i = num_val -j -1
  signals_val1 = [eeg1_val[i].reshape(1,1500),eeg2_val[i].reshape(1,1500),
                 eeg3_val[i].reshape(1,1500),eeg4_val[i].reshape(1,1500),eeg5_val[i].reshape(1,1500),
                 eeg6_val[i].reshape(1,1500),eeg7_val[i].reshape(1,1500)]
  #Preprocessing
  Pval = proc(signals_val1)
  
  #Extracting features
  signals_val = [accX_val[i].reshape(1,300),accY_val[i].reshape(1,300),
                 accZ_val[i].reshape(1,300),pulse_val[i].reshape(1,300)]
  for j in range(7):
      for k in range(3):
          signals_val.append(Pval[k][j])
  Dval, pca = feat_extract(signals_val,-1)
  
  #predicting the test
  Y_val[i] = model_final.predict(Dval)
 
for i in tqdm(range(int(num_val/pack))):
  
  signals_val1 = [eeg1_val[i*pack:(i+1)*pack].reshape(pack,1500),eeg2_val[i*pack:(i+1)*pack].reshape(pack,1500),
                 eeg3_val[i*pack:(i+1)*pack].reshape(pack,1500),eeg4_val[i*pack:(i+1)*pack].reshape(pack,1500),eeg5_val[i*pack:(i+1)*pack].reshape(pack,1500),
                 eeg6_val[i*pack:(i+1)*pack].reshape(pack,1500),eeg7_val[i*pack:(i+1)*pack].reshape(pack,1500)]
  #Preprocessing
  Pval = proc(signals_val1)
  
  #Extracting features
  signals_val = [accX_val[i*pack:(i+1)*pack].reshape(pack,300),accY_val[i*pack:(i+1)*pack].reshape(pack,300),
                 accZ_val[i*pack:(i+1)*pack].reshape(pack,300),pulse_val[i*pack:(i+1)*pack].reshape(pack,300)]
  for j in range(7):
      for k in range(3):
          signals_val.append(Pval[k][j])
  Dval, pca = feat_extract(signals_val,-1)
  
  #predicting the test
  Y_val[i*pack:(i+1)*pack] = model_final.predict(Dval)


print("Prediciton time: ",time.time()-t1)

Training time:  23.848984003067017


100%|██████████| 374/374 [07:28<00:00,  1.15s/it]

Prediciton time:  487.72379517555237


In [0]:
Y_val = Y_val.astype(int)

In [0]:
#produces the CSV file to submit

num_samples_val = Y_val.shape[0]
with open('tosubmit.csv', 'w', newline = '') as csvfile:
    writer = csv.writer(csvfile, dialect = "excel")
    writer.writerow(["id","sleep_stage"])
    for i in range(num_samples_val):
        writer.writerow([i,Y_val[i]])